In [1]:
import os
from typing import List, Any
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType, ChiralType
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import random
import joblib
import math
import statistics

In [2]:
randomNumber = 42

In [3]:
def get_file_list(catalogue: str, t_l=None):
    f_l = []
    f_d = dict()
    if t_l is None:
        t_l = [
            'xvg', 'gro', 'pdb', 'log', 'out', 'dat', 'xyz', 'csv', 'xlsx', 'xls', 'txt',
            'png', 'jpg', 'jpeg', 'gif', 'bmp', 'tif', 'tiff', 'svg', 'psd', 'ai', 'eps',
            'raw', 'cr2', 'nef', 'orf', 'rw2', 'arw', 'dng', 'raf', 'srw', 'sr2',
            'tif', 'tiff', 'tif', 'tiff', 'tif', 'tiff', 'tif', 'tiff',
        ]
    elif isinstance(t_l, str):
        t_l = [t_l]
    catalogue = os.path.abspath(catalogue)
    f_l = os.listdir(catalogue)
    if f_l:
        for t in t_l:
            f_d[t] = [os.path.join(catalogue, f) for f in f_l if t in f]
    return f_d

In [4]:
def get_new_dict(file_dir: str, suffix: list):
    file_List = get_file_list(file_dir, suffix)
    file_List = file_List['csv']
    newDict = {
        '001': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '010': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '011': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '100': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '101': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '110': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
        '111': {
            'LR_Top2': [], 'LR_AUC': [], 'LR_MCC': [],
            'NB_Top2': [], 'NB_AUC': [], 'NB_MCC': [],
            'SVM_Top2': [], 'SVM_AUC': [], 'SVM_MCC': [],
            'DT_Top2': [], 'DT_AUC': [], 'DT_MCC': [],
            'RF_Top2': [], 'RF_AUC': [], 'RF_MCC': [],
        },
    }
    for file_path in file_List:
        df = pd.read_csv(file_path)
        # print("df=", df)
        for index, row in df.iterrows():
            binValue = str(
                f"{row['Atom-type']}" +
                f"{row['Atom-based']}" +
                f"{row['Topol-based']}"
            )
            if row['Performa nce metrics'] == 'Top2':
                newDict[binValue]['LR_Top2'].append(df['LR'][index])
                newDict[binValue]['NB_Top2'].append(df['NB'][index])
                newDict[binValue]['SVM_Top2'].append(df['SVM'][index])
                newDict[binValue]['DT_Top2'].append(df['DT'][index])
                newDict[binValue]['RF_Top2'].append(df['RF'][index])
    
            if row['Performa nce metrics'] == 'AUC':
                newDict[binValue]['LR_AUC'].append(df['LR'][index])
                newDict[binValue]['NB_AUC'].append(df['NB'][index])
                newDict[binValue]['SVM_AUC'].append(df['SVM'][index])
                newDict[binValue]['DT_AUC'].append(df['DT'][index])
                newDict[binValue]['RF_AUC'].append(df['RF'][index])
            if row['Performa nce metrics'] == 'MCC':
                newDict[binValue]['LR_MCC'].append(df['LR'][index])
                newDict[binValue]['NB_MCC'].append(df['NB'][index])
                newDict[binValue]['SVM_MCC'].append(df['SVM'][index])
                newDict[binValue]['DT_MCC'].append(df['DT'][index])
                newDict[binValue]['RF_MCC'].append(df['RF'][index])
    return newDict

In [5]:
def get_new_data_Dict(file_dir, suffix, save_dir, index):
    newDict = get_new_dict(file_dir, suffix)
    new_data_dict = pd.DataFrame(
    columns=[
        'Atom-type', 'Atom-based', 'Topol-based',
        'Performa nce metrics',
        'LR_mean', 'LR_variance',
        'NB_mean', 'NB_variance',
        'SVM_mean', 'SVM_variance',
        'DT_mean', 'DT_variance',
        'RF_mean', 'RF_variance'
    ])
    Atom_type = []
    Atom_based = []
    Topol_based = []
    Performa_nce_metrics = []
    LR_mean = []
    LR_variance = []
    NB_mean = []
    NB_variance = []
    SVM_mean = []
    SVM_variance = []
    DT_mean = []
    DT_variance = []
    RF_mean = []
    RF_variance = []

    for i in range(8):
        if i == 0:
            continue
        else:
            binNumber = bin(i)[2:].zfill(3)
            old_traits_ = list(binNumber)
            traits_ = ''.join(old_traits_)
            Atom_type.extend([old_traits_[0], old_traits_[0], old_traits_[0]])
            Atom_based.extend([old_traits_[1], old_traits_[1], old_traits_[1]])
            Topol_based.extend([old_traits_[2], old_traits_[2], old_traits_[2]])
            Performa_nce_metrics.append('Top2')
            Performa_nce_metrics.append('AUC')
            Performa_nce_metrics.append('MCC')
            newDict[traits_]['LR_mean'] = [
                round(statistics.mean(newDict[traits_]['LR_Top2']), 1),
                round(statistics.mean(newDict[traits_]['LR_AUC']), 3),
                round(statistics.mean(newDict[traits_]['LR_MCC']), 3),
            ]
            LR_mean.extend(newDict[traits_]['LR_mean'])
            newDict[traits_]['LR_variance'] = [
                round(statistics.variance(newDict[traits_]['LR_Top2']), 1),
                round(statistics.variance(newDict[traits_]['LR_AUC']), 3),
                round(statistics.variance(newDict[traits_]['LR_MCC']), 3),
            ]
            LR_variance.extend(newDict[traits_]['LR_variance'])
            newDict[traits_]['NB_mean'] = [
                round(statistics.mean(newDict[traits_]['NB_Top2']), 1),
                round(statistics.mean(newDict[traits_]['NB_AUC']), 3),
                round(statistics.mean(newDict[traits_]['NB_MCC']), 3),
            ]
            NB_mean.extend(newDict[traits_]['NB_mean'])
            newDict[traits_]['NB_variance'] = [
                round(statistics.variance(newDict[traits_]['NB_Top2']), 1),
                round(statistics.variance(newDict[traits_]['NB_AUC']), 3),
                round(statistics.variance(newDict[traits_]['NB_MCC']), 3),
            ]
            NB_variance.extend(newDict[traits_]['NB_variance'])
            newDict[traits_]['SVM_mean'] = [
                round(statistics.mean(newDict[traits_]['SVM_Top2']), 1),
                round(statistics.mean(newDict[traits_]['SVM_AUC']), 3),
                round(statistics.mean(newDict[traits_]['SVM_MCC']), 3),
            ]
            SVM_mean.extend(newDict[traits_]['SVM_mean'])
    
            newDict[traits_]['SVM_variance'] = [
                round(statistics.variance(newDict[traits_]['SVM_Top2']), 1),
                round(statistics.variance(newDict[traits_]['SVM_AUC']), 3),
                round(statistics.variance(newDict[traits_]['SVM_MCC']), 3),
            ]
            SVM_variance.extend(newDict[traits_]['SVM_variance'])
            newDict[traits_]['DT_mean'] = [
                round(statistics.mean(newDict[traits_]['DT_Top2']), 1),
                round(statistics.mean(newDict[traits_]['DT_AUC']), 3),
                round(statistics.mean(newDict[traits_]['DT_MCC']), 3),
            ]
            DT_mean.extend(newDict[traits_]['DT_mean'])
            newDict[traits_]['DT_variance'] = [
                round(statistics.variance(newDict[traits_]['DT_Top2']), 1),
                round(statistics.variance(newDict[traits_]['DT_AUC']), 3),
                round(statistics.variance(newDict[traits_]['DT_MCC']), 3),
            ]
            DT_variance.extend(newDict[traits_]['DT_variance'])
            newDict[traits_]['RF_mean'] = [
                round(statistics.mean(newDict[traits_]['RF_Top2']), 1),
                round(statistics.mean(newDict[traits_]['RF_AUC']), 3),
                round(statistics.mean(newDict[traits_]['RF_MCC']), 3),
            ]
            RF_mean.extend(newDict[traits_]['RF_mean'])
            newDict[traits_]['RF_variance'] = [
                round(statistics.variance(newDict[traits_]['RF_Top2']), 1),
                round(statistics.variance(newDict[traits_]['RF_AUC']), 3),
                round(statistics.variance(newDict[traits_]['RF_MCC']), 3),
            ]
            RF_variance.extend(newDict[traits_]['RF_variance'])

    new_data_dict['Atom-type'] = Atom_type
    new_data_dict['Atom-based'] = Atom_based
    new_data_dict['Topol-based'] = Topol_based
    new_data_dict['Performa nce metrics'] = Performa_nce_metrics
    max_len = max(
        len(LR_mean), len(LR_variance),
        len(NB_mean), len(NB_variance),
        len(SVM_mean), len(SVM_variance),
        len(DT_mean), len(DT_variance),
        len(RF_mean), len(RF_variance))
    new_data_dict['LR_mean'] = LR_mean if len(LR_mean) == max_len else [None for i in range(max_len)]
    new_data_dict['LR_variance'] = LR_variance if len(LR_variance) == max_len else [None for i in range(max_len)]
    new_data_dict['NB_mean'] = NB_mean if len(NB_mean) == max_len else [None for i in range(max_len)]
    new_data_dict['NB_variance'] = NB_variance if len(NB_variance) == max_len else [None for i in range(max_len)]
    new_data_dict['SVM_mean'] = SVM_mean if len(SVM_mean) == max_len else [None for i in range(max_len)]
    new_data_dict['SVM_variance'] = SVM_variance if len(SVM_variance) == max_len else [None for i in range(max_len)]
    new_data_dict['DT_mean'] = DT_mean if len(DT_mean) == max_len else [None for i in range(max_len)]
    new_data_dict['DT_variance'] = DT_variance if len(DT_variance) == max_len else [None for i in range(max_len)]
    new_data_dict['RF_mean'] = RF_mean if len(RF_mean) == max_len else [None for i in range(max_len)]
    new_data_dict['RF_variance'] = RF_variance if len(RF_variance) == max_len else [None for i in range(max_len)]
    save_path = os.path.join(save_dir, f'mean_vs_variance{index}.csv')
    print('save_path = ', save_path)
    new_data_dict.to_csv(save_path)

# 1 Calculate the score

## 1.1 Calculate the Top2

In [6]:
def find_topk(a, k, axis=-1, largest=True, sorted_=True):
    if axis is None:
        axis_size = a.size
    else:
        axis_size = a.shape[axis]
    assert 1 <= k <= axis_size

    a = np.asanyarray(a)
    if largest:
        index_array = np.argpartition(a, axis_size-k, axis=axis)
        topk_indices = np.take(index_array, -np.arange(k)-1, axis=axis)
    else:
        index_array = np.argpartition(a, k-1, axis=axis)
        topk_indices = np.take(index_array, np.arange(k), axis=axis)
    topk_values = np.take_along_axis(a, topk_indices, axis=axis)
    if sorted_:
        sorted_indices_in_topk = np.argsort(topk_values, axis=axis)
        if largest:
            sorted_indices_in_topk = np.flip(sorted_indices_in_topk, axis=axis)
        sorted_topk_values = np.take_along_axis(
            topk_values, sorted_indices_in_topk, axis=axis)
        sorted_topk_indices = np.take_along_axis(
            topk_indices, sorted_indices_in_topk, axis=axis)
        return sorted_topk_values, sorted_topk_indices
    return topk_values, topk_indices

# 2.1 Set up the feature

In [7]:
def _get_node_features(
        mol,
        traits=None,
):
    if traits is None:
        traits = [True, True, True]
    all_node_feats = []
    identity = {
        'C': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'N': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        'O': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        'F': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        'P': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        'S': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        'Cl': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        'Br': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        'I': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        'other': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    }
    for atom in mol.GetAtoms():
        node_feats = []
        # atom number
        idx = atom.GetIdx()

        # Atom-type
        if traits[0]:
            # atom type one-hot 10
            node_feats.extend(identity.get(atom.GetSymbol(), [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

        # Atom-based
        if traits[1]:
            # implicit valence
            node_feats.append(atom.GetImplicitValence())
            # formal charge
            node_feats.append(atom.GetFormalCharge())
            # radical electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # aromatic 0 or 1
            if atom.GetIsAromatic():
                node_feats.append(1)
            else:
                node_feats.append(0)

            # chirality
            chirality = atom.GetChiralTag()
            temp = None
            if chirality == ChiralType.CHI_TETRAHEDRAL_CCW: temp = [1, 0, 0, 0]
            if chirality == ChiralType.CHI_TETRAHEDRAL_CW: temp = [0, 1, 0, 0]
            if chirality == ChiralType.CHI_OTHER: temp = [0, 0, 1, 0]
            if chirality == ChiralType.CHI_UNSPECIFIED: temp = [0, 0, 0, 1]
            node_feats.extend(temp)
            # hybridization
            hybridization = atom.GetHybridization()
            tmp = None
            if hybridization == HybridizationType.S: tmp = [1, 0, 0, 0, 0, 0, 0, 0]
            if hybridization == HybridizationType.SP: tmp = [0, 1, 0, 0, 0, 0, 0, 0]
            if hybridization == HybridizationType.SP2: tmp = [0, 0, 1, 0, 0, 0, 0, 0]
            if hybridization == HybridizationType.SP3: tmp = [0, 0, 0, 1, 0, 0, 0, 0]
            if hybridization == HybridizationType.SP3D: tmp = [0, 0, 0, 0, 1, 0, 0, 0]
            if hybridization == HybridizationType.SP3D2: tmp = [0, 0, 0, 0, 0, 1, 0, 0]
            if hybridization == HybridizationType.OTHER: tmp = [0, 0, 0, 0, 0, 0, 1, 0]
            if hybridization == HybridizationType.UNSPECIFIED: tmp = [0, 0, 0, 0, 0, 0, 0, 1]
            node_feats.extend(tmp)

        # Topol-based
        if traits[2]:
            # degree
            node_feats.append(atom.GetDegree())
            # hydrogen
            node_feats.append(atom.GetTotalNumHs())
            # ring status 0 or 1
            if atom.IsInRing():
                node_feats.append(1)
            else:
                node_feats.append(0)
        # Append node features to matrix
        all_node_feats.append(node_feats)

    all_node_feats = np.asarray(all_node_feats, dtype=np.float64)
    return all_node_feats

## 2.2 Get feature labels

In [8]:
def _get_labels(mol):
    _y = []
    som = ['PRIMARY_SOM_1A2', 'PRIMARY_SOM_2A6', 'PRIMARY_SOM_2B6', 'PRIMARY_SOM_2C8', 'PRIMARY_SOM_2C9',
           'PRIMARY_SOM_2C19', 'PRIMARY_SOM_2D6', 'PRIMARY_SOM_2E1', 'PRIMARY_SOM_3A4',
           'SECONDARY_SOM_1A2', 'SECONDARY_SOM_2A6', 'SECONDARY_SOM_2B6', 'SECONDARY_SOM_2C8', 'SECONDARY_SOM_2C9',
           'SECONDARY_SOM_2C19', 'SECONDARY_SOM_2D6', 'SECONDARY_SOM_2E1', 'SECONDARY_SOM_3A4',
           'TERTIARY_SOM_1A2', 'TERTIARY_SOM_2A6', 'TERTIARY_SOM_2B6', 'TERTIARY_SOM_2C8', 'TERTIARY_SOM_2C9',
           'TERTIARY_SOM_2C19', 'TERTIARY_SOM_2D6', 'TERTIARY_SOM_2E1', 'TERTIARY_SOM_3A4'
           ]
    result = []
    for k in som:
        try:
            _res = mol.GetProp(k)
            if ' ' in _res:
                res = _res.split(' ')
                for s in res:
                    result.append(int(s))
            else:
                result.append(int(_res))
        except KeyError:
            pass

    for data in result:
        _y.append(data)
    _y = list(set(_y))

    y = np.zeros(len(mol.GetAtoms()))
    for i in _y:
        y[i - 1] = 1
    return y

## 2.3 Obtain molecular data

In [9]:
def get_mol_data(
        file_path: str = None,
        train_number: int = 544,
        index: int = 0
):
    raws = Chem.SDMolSupplier(file_path)
    mols = [mol for mol in raws]
    # Set up a random seed to ensure the results are reproducible.
    random.seed(randomNumber)
    # Shuffle the order.
    random.shuffle(mols)
    mols_train = mols[:train_number]
    mols_test = mols[train_number:]
    return mols_train, mols_test

## 2.4 Set up collection data

In [10]:
def get_set_data(mols_train, mols_test, traits=None):
    train_set = []
    test_set = []
    te_indices = []
    for mol in mols_train:
        mol_feature = _get_node_features(mol=mol, traits=traits)
        label = _get_labels(mol)
        train_set.append((mol_feature, label))
    for mol in mols_test:
        mol_feature = _get_node_features(mol=mol, traits=traits)
        label = _get_labels(mol)
        test_set.append((mol_feature, label))
        te_indices.append(len(label))
    return train_set, test_set, te_indices

## 2.5 Set up training and test data

In [11]:
def get_train_and_test(train_set, test_set, te_indices):
    tr_all = 0  # all training atoms
    tr_indices = []  # molecule nums
    y_train = []  # label
    for feature, label in train_set:
        tr_all += feature.shape[0]
        tr_indices.append(len(label))
        y_train.extend(label)

    x_train = np.zeros((tr_all, train_set[0][0].shape[1]))
    pre = 0
    for i in range(len(tr_indices)):
        if i == 0:
            x_train[:tr_indices[i], :] = train_set[i][0]
        else:
            j = i - 1
            pre += tr_indices[j]
            x_train[pre:pre + tr_indices[i], :] = train_set[i][0]
    y_train = np.array(y_train)

    # Stitching matrix
    te_all = 0
    y_test = []
    for feature, label in test_set:
        te_all += feature.shape[0]
        y_test.extend(label)
    x_test = np.zeros((te_all, train_set[0][0].shape[1]))
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            x_test[:te_indices[i], :] = test_set[i][0]
        else:
            j = i - 1
            pre += te_indices[j]
            x_test[pre:pre + te_indices[i], :] = test_set[i][0]
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test,

# 3 Machine learning models

## 3.1 Logistic regression

In [12]:
def ml_lr(x_train, y_train, x_test, y_test, test_set, te_indices, index: int, save_model_path: str):
    name = os.path.basename(save_model_path).split('.')[0]
    
    if os.path.exists(save_model_path):
        clf = joblib.load(save_model_path)
    else:
        max_iter = 20 + 5 * index
        # C = 6 + index
        clf = LogisticRegression(
            C=10,
            solver='sag',
            random_state=randomNumber,
            class_weight='balanced',
            max_iter=max_iter,
            n_jobs=-1
        ).fit(x_train, y_train)
        joblib.dump(clf, save_model_path)
    preds = clf.predict(x_test)
    logits = clf.predict_proba(x_test)
    top2 = 0
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            y_preds = logits[:te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
        else:
            j = i - 1
            pre += te_indices[j]
            y_preds = logits[pre:pre + te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
    return {
        'Top2': round(top2 *100 / len(test_set), 3),
        'AUC': round(roc_auc_score(y_test, preds), 3),
        'MCC': round(matthews_corrcoef(y_test, preds), 3),
    }

## 3.2 Naive Bayes

In [13]:
def ml_nb(x_train, y_train, x_test, y_test, test_set, te_indices, index: int, save_model_path: str):
    if os.path.exists(save_model_path):
        clf = joblib.load(save_model_path)
    else:
        var_smoothing = 0.0000000001 * math.pow(10, index+1)
        clf = GaussianNB(var_smoothing=var_smoothing)
        clf.fit(x_train, y_train)
        joblib.dump(clf, save_model_path)
    preds = clf.predict(x_test)
    logits = clf.predict_proba(x_test)
    top2 = 0
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            y_preds = logits[:te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
        else:
            j = i - 1
            pre += te_indices[j]
            y_preds = logits[pre:pre + te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
    return {
        'Top2': round(top2 *100 / len(test_set), 3),
        'AUC': round(roc_auc_score(y_test, preds), 3),
        'MCC': round(matthews_corrcoef(y_test, preds), 3),
    }

## 3.3 Support vector machines

In [14]:
def ml_svm(x_train, y_train, x_test, y_test, test_set, te_indices, index: int, save_model_path: str):
    if os.path.exists(save_model_path):
        clf = joblib.load(save_model_path)
    else:
        C = 0.06 + 0.01 * index
        clf = SVC(
            C=C,
            degree=2,
            gamma='scale',
            kernel='linear',
            random_state=randomNumber,
            class_weight='balanced',
            probability=True).fit(x_train, y_train)
    joblib.dump(clf, save_model_path)
    preds = clf.predict(x_test)
    logits = clf.predict_proba(x_test)
    top2 = 0
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            y_preds = logits[:te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
        else:
            j = i - 1
            pre += te_indices[j]
            y_preds = logits[pre:pre + te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
    return {
        'Top2': round(top2 *100 / len(test_set), 3),
        'AUC': round(roc_auc_score(y_test, preds), 3),
        'MCC': round(matthews_corrcoef(y_test, preds), 3),
    }

## 3.4 Decision tree

In [15]:
def ml_dt(x_train, y_train, x_test, y_test, test_set, te_indices, index: int, save_model_path: str):
    if os.path.exists(save_model_path):
        clf = joblib.load(save_model_path)
    else:
        max_depths = 1 + index
        clf = DecisionTreeClassifier(
            random_state=randomNumber,
            class_weight='balanced',
            criterion='gini',
            max_depth=max_depths,
            splitter='best'
        ).fit(x_train, y_train)
        joblib.dump(clf, save_model_path)
    preds = clf.predict(x_test)
    logits = clf.predict_proba(x_test)
    top2 = 0
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            y_preds = logits[:te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
        else:
            j = i - 1
            pre += te_indices[j]
            y_preds = logits[pre:pre + te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
    return {
        'Top2': round(top2 *100 / len(test_set), 3),
        'AUC': round(roc_auc_score(y_test, preds), 3),
        'MCC': round(matthews_corrcoef(y_test, preds), 3),
    }

## 3.5 Random forest

In [16]:
def ml_rf(x_train, y_train, x_test, y_test, test_set, te_indices, index: int, save_model_path: str):
    if os.path.exists(save_model_path):
        clf = joblib.load(save_model_path)
    else:
        n_estimators = 50 + 15 * index
        clf = RandomForestClassifier(
            random_state=randomNumber,
            n_estimators=n_estimators,
            max_depth=20,
            criterion='entropy',
            class_weight='balanced'
        ).fit(x_train, y_train)
        joblib.dump(clf, save_model_path)
    preds = clf.predict(x_test)
    logits = clf.predict_proba(x_test)
    top2 = 0
    pre = 0
    for i in range(len(te_indices)):
        if i == 0:
            # 取出第一列，
            y_preds = logits[:te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
        else:
            j = i - 1
            pre += te_indices[j]
            y_preds = logits[pre:pre + te_indices[i], :][:, 1]
            topk_values, topk_indices = find_topk(y_preds, 2)
            for a in range(len(test_set[i][1])):
                if test_set[i][1][a] == 1 and a in topk_indices:
                    top2 += 1
                    break
    return {
        'Top2': round(top2 *100 / len(test_set), 3),
        'AUC': round(roc_auc_score(y_test, preds), 3),
        'MCC': round(matthews_corrcoef(y_test, preds), 3),
    }

In [17]:
def main(file_path, modelsDir, resultDir, newResultsDir, number: int = 1):
    if not os.path.exists(modelDir): os.mkdir(modelDir)
    if not os.path.exists(resultsDir): os.mkdir(resultsDir)
    if not os.path.exists(newResultsDir): os.mkdir(newResultsDir)
    for index_ in range(number):
        molsTrain, molsTest = get_mol_data(filepath, train_number=544)
        newModelDir = os.path.join(modelDir, os.path.basename(modelDir) + str(index_))
        if not os.path.exists(newModelDir):
            os.mkdir(newModelDir)
        DataFrameSet = pd.DataFrame(
            columns=[
                'Atom-type', 'Atom-based', 'Topol-based',
                'Performa nce metrics',
                'LR', 'NB', 'SVM', 'DT', 'RF'])
        Atom_type = []
        Atom_based = []
        Topol_based = []
        Performa_nce_metrics = []
        LR = []
        NB = []
        SVM = []
        DT = []
        RF = []
        mix = 8
        if os.path.exists(os.path.join(resultsDir, f'result_{index_}.csv')):
            continue
        for i in tqdm(range(mix), total=mix, desc=f'Number of trainings[{number}/{index_}]'):
            if i == 0:
                continue
            # elif i == 4:
            #     continue
            else:
                binNumber = bin(i)[2:].zfill(3)
                traits_ = list(binNumber)
                traits_ = [int(j) for j in traits_]
                Atom_type.extend([traits_[0], traits_[0], traits_[0]])
                Atom_based.extend([traits_[1], traits_[1], traits_[1]])
                Topol_based.extend([traits_[2], traits_[2], traits_[2]])
                trainSet, testSet, teIndices = get_set_data(molsTrain, molsTest, traits=traits_)
                xTrain, yTrain, xTest, yTest = get_train_and_test(trainSet, testSet, teIndices)
                Performa_nce_metrics.append('Top2')
                Performa_nce_metrics.append('AUC')
                Performa_nce_metrics.append('MCC')
                # print('Logistic regression prediction...')
                newSaveDir = os.path.join(newModelDir, 'LR')
                if not os.path.exists(newSaveDir): os.mkdir(newSaveDir)
                newSavePath = os.path.join(newSaveDir, f'LR_{str(binNumber)}.joblib')
                lrScores = ml_lr(xTrain, yTrain, xTest, yTest, testSet, teIndices, index_, newSavePath)
                for key, item in lrScores.items():
                    LR.append(item)
                # print('Naive Bayes predicts...')
                newSaveDir = os.path.join(newModelDir, 'NB')
                if not os.path.exists(newSaveDir): os.mkdir(newSaveDir)
                newSavePath = os.path.join(newSaveDir, f'NB_{str(binNumber)}.joblib')
                nbScores = ml_nb(xTrain, yTrain, xTest, yTest, testSet, teIndices, index_, newSavePath)
                for key, item in nbScores.items():
                    NB.append(item)
                # print('Decision tree prediction...')
                newSaveDir = os.path.join(newModelDir, 'DT')
                if not os.path.exists(newSaveDir): os.mkdir(newSaveDir)
                newSavePath = os.path.join(newSaveDir, f'DT_{str(binNumber)}.joblib')
                dtScores = ml_dt(xTrain, yTrain, xTest, yTest, testSet, teIndices, index_, newSavePath)
                for key, item in dtScores.items():
                    DT.append(item)
                # print('Random forest predictions...')
                newSaveDir = os.path.join(newModelDir, 'RF')
                if not os.path.exists(newSaveDir): os.mkdir(newSaveDir)
                newSavePath = os.path.join(newSaveDir, f'RF_{str(binNumber)}.joblib')
                rfScores = ml_rf(xTrain, yTrain, xTest, yTest, testSet, teIndices, index_, newSavePath)
                for key, item in rfScores.items():
                    RF.append(item)
                # print('Support vector machine prediction...')
                newSaveDir = os.path.join(newModelDir, 'SVM')
                if not os.path.exists(newSaveDir): os.mkdir(newSaveDir)
                newSavePath = os.path.join(newSaveDir, f'SVM_{str(binNumber)}.joblib')
                svmScores = ml_svm(xTrain, yTrain, xTest, yTest, testSet, teIndices, index_, newSavePath)
                for key, item in svmScores.items():
                    SVM.append(item)
        DataFrameSet['Atom-type'] = Atom_type
        DataFrameSet['Atom-based'] = Atom_based
        DataFrameSet['Topol-based'] = Topol_based
        DataFrameSet['Performa nce metrics'] = Performa_nce_metrics
        max_len = max(len(LR), len(NB), len(SVM), len(DT), len(RF))
        DataFrameSet['LR'] = LR if len(LR) == max_len else [None for i in range(max_len)]
        DataFrameSet['NB'] = NB if len(NB) == max_len else [None for i in range(max_len)]
        DataFrameSet['SVM'] = SVM if len(SVM) == max_len else [None for i in range(max_len)]
        DataFrameSet['DT'] = DT if len(DT) == max_len else [None for i in range(max_len)]
        DataFrameSet['RF'] = RF if len(RF) == max_len else [None for i in range(max_len)]
        DataFrameSet.to_csv(os.path.join(resultsDir, f'result_{index_}.csv'))
    # for index_ in range(number):
    get_new_data_Dict(resultsDir, 'csv', newResultsDir, 3)
        

In [18]:
filepath = '/home/dldx/DATA/JXDATA/GitHubProjects/GraphCySoM/datasets/merged.sdf'
modelDir = '/home/dldx/DATA/JXDATA/GitHubProjects/GraphCySoM/datasets/models'
resultsDir = '/home/dldx/DATA/JXDATA/GitHubProjects/GraphCySoM/datasets/results'
newResultsDir = '/home/dldx/DATA/JXDATA/GitHubProjects/GraphCySoM/datasets/mean_vs_variance'

In [19]:
main(filepath, modelDir, resultsDir, newResultsDir, number=10)

save_path =  /home/dldx/DATA/JXDATA/GitHubProjects/GraphCySoM/datasets/mean_vs_variance/mean_vs_variance3.csv
